In [1]:
import pandas as pd
from meetup import get_meetup_events
from mhub import get_mhub_events
from luma import get_luma_group_events
from common import upload_to_gsheets, get_gsheet_df, create_event_markdown, create_discord_list
from datetime import datetime

# TODO: Get 1871 Events
# TODO: Get lu.ma users Events
# TODO: Fix luma
# TODO: Get Eventbrite Events
# TODO: Figure out the list of others that need to be scraped
# TODO: Figure out a way to make this format nicely for pasting into slack/discord
# TODO: Make a website/calendar
# TODO: Filter non-Chicago events from meetup groups (e.g. https://www.meetup.com/pydatachi/events/303000483/)
# TODO: Split slack messages into multiple markdowns to make life easier
# TODO: Throw this into ChatGPT to simplify the list?
# TODO: Add Organization to Markdown

In [2]:
organization_df = get_gsheet_df("Organizations")

In [3]:
meetup_groups = organization_df['Meetup'].tolist()
meetup_groups = list(set([x for x in meetup_groups if len(x) > 0]))

luma_groups = organization_df['LuMa'].tolist()
luma_groups = list(set([x for x in luma_groups if len(x) > 0]))

In [4]:
organization_df

,,Organization,Category,Subcategory,Active,Scraper Configured,Website,Event Page,Description,Last In-Person Event,...,LinkedIn,Slack,Discord,Meetup,LuMa,EventBrite,YouTube,GitHub,Facebook,Mastodon
0,1,Chicago Python User Group,Tech Organizations/Groups/Communities,General/Others,TRUE,,https://www.chipy.org/,https://www.meetup.com/_ChiPy_/,"Founded in 2003, Chicago Python User Group is ...",,...,,https://www.chipy.org/pages/slack/,,https://www.meetup.com/_ChiPy_/,,,https://www.youtube.com/channel/UCT372EAC1orBO...,https://github.com/chicagopython/chipy.org,,
1,1,Codecademy + 100Devs Chicago Meetup,Tech Organizations/Groups/Communities,General/Others,FALSE,,https://www.meetup.com/100devs-meetup-group/,https://www.meetup.com/100devs-meetup-group/,"Hello!\nWelcome to 312Coders, a vibrant coding...",,...,,,,https://www.meetup.com/100devs-meetup-group/,,,,,,
2,1,312Coders,Tech Organizations/Groups/Communities,Social,TRUE,,https://www.meetup.com/312-coders/,https://www.meetup.com/312-coders/,"Hello!\nWelcome to 312Coders, a vibrant coding...",,...,,,,https://www.meetup.com/312-coders/,,,,,,
3,1,84.51 Science and Technology Solutions - Chicago,,,TRUE,,https://www.meetup.com/84-51-science-and-techn...,https://www.meetup.com/84-51-science-and-techn...,The purpose of this group is to share exciting...,,...,,,,https://www.meetup.com/84-51-science-and-techn...,,,,,,
4,,8th Light University,,,TRUE,,https://www.meetup.com/8th-light-university/,https://www.meetup.com/8th-light-university/,8th Light University is a virtual event series...,,...,,,,https://www.meetup.com/8th-light-university/,,,https://www.youtube.com/@8thLightInc,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,1,University of Chicago,Universities,,TRUE,,https://www.uchicago.edu/education-and-researc...,,,,...,,,,,,,,,,
227,1,WiSTEM - Focus on women-led startups,Incubators/Accelerators,,TRUE,,http://www.1871.com/wistem/,,,,...,,,,,,,,,,
228,2,Women in Data Science,Identity-Focused,,TRUE,,https://widschicago.org/,,WiDS Chicago is independently organized by loc...,,...,,,,,,,,,,
229,,Product Club at Illinois Tech,,,,,https://lu.ma/user/pmclubiit,https://lu.ma/user/pmclubiit,,,...,,,,,https://lu.ma/user/pmclubiit,,,,,


In [5]:
try:
    meetup_df = get_meetup_events(meetup_groups, sleep_time=2, verbose=True, debug=True)
except Exception as e:
    print('Failed to get Meetup events')
    print(e)
try:
    mhub_df = get_mhub_events()
except Exception as e:
    print('Failed to get MHub events')
    print(e)
try:
    luma_df = get_luma_group_events(luma_groups, sleep_time=1, verbose=False, debug=True)
except Exception as e:
    print('Failed to get LuMa events')
    print(e)

0/107 -  https://www.meetup.com/nhi-collective-chicago/events/
1/107 -  https://www.meetup.com/_ChiPy_/events/
0/10 -  https://www.meetup.com/_chipy_/events/302396651/
1/10 -  https://www.meetup.com/_chipy_/events/302396652/
2/10 -  https://www.meetup.com/_chipy_/events/302396653/
3/10 -  https://www.meetup.com/_chipy_/events/302396654/
4/10 -  https://www.meetup.com/_chipy_/events/302396656/
5/10 -  https://www.meetup.com/_chipy_/events/vxxpjtyhclbkb/
6/10 -  https://www.meetup.com/_chipy_/events/vxxpjtyhcmbgb/
7/10 -  https://www.meetup.com/_chipy_/events/vxxpjtyhcnbdb/
8/10 -  https://www.meetup.com/_chipy_/events/vxxpjtyhcpbjb/
9/10 -  https://www.meetup.com/_chipy_/events/vxxpjtyhcqbgb/
2/107 -  https://www.meetup.com/js-chi/events/
0/4 -  https://www.meetup.com/js-chi/events/305185313/
1/4 -  https://www.meetup.com/js-chi/events/306257990/
2/4 -  https://www.meetup.com/js-chi/events/306257996/
3/4 -  https://www.meetup.com/js-chi/events/306258006/
3/107 -  https://www.meetup.com/

In [6]:
# combine the dataframes
try:
    combined_df = pd.concat([meetup_df, mhub_df, luma_df], ignore_index=True)
except Exception as e:
    print(e)
    combined_df = pd.concat([mhub_df, meetup_df], ignore_index=True)

combined_df

name 'luma_df' is not defined


,eventName,eventURL,eventStartTime,eventendTime,eventVenueName,eventAddress,eventCity,eventState,groupName,eventGoogleMaps,event_description,datetimeScraped
0,mHUB HardTech Summit,https://www.mhubchicago.com/events/mhub-hardte...,2025-03-06 13:30:00,,mHUB,1623 W Fulton St,Chicago,IL,mHUB,https://maps.google.com/maps?saddr=&daddr=mHUB...,Discover the Future with HardTech Innovation T...,2025-03-03 00:22:01.001777+00:00
1,mHUB Fourth Revolution Awards,https://www.mhubchicago.com/events/mhub-fourth...,2025-03-06 19:00:00,,Venue West,"Venue West, North Paulina Street, Chicago, IL,...",None,None,mHUB,https://maps.google.com/maps?saddr=&daddr=Venu...,Celebrate the Midwest’s Hardtech and Industria...,2025-03-03 00:22:03.146467+00:00
2,Build Targeted Lists and Test Your Value Prop ...,https://www.mhubchicago.com/events/build-targe...,2025-03-10 12:00:00,,mHUB,1623 W Fulton St,Chicago,IL,mHUB,https://maps.google.com/maps?saddr=&daddr=mHUB...,mindHUB Sessions: Member-Led WorkshopsBuild Ta...,2025-03-03 00:22:05.226122+00:00
3,Financial Modeling and Go to Market Terminology,https://www.mhubchicago.com/events/financial-m...,2025-03-12 13:00:00,,mHUB,1623 W Fulton St,Chicago,IL,mHUB,https://maps.google.com/maps?saddr=&daddr=mHUB...,Join us for a workshop where we'll delve into ...,2025-03-03 00:22:07.307114+00:00
4,Beyond the Pitch - Mastering Strategic Communi...,https://www.mhubchicago.com/events/beyond-the-...,2025-03-13 13:30:00,,mHUB,1623 W Fulton St,Chicago,IL,mHUB,https://maps.google.com/maps?saddr=&daddr=mHUB...,Beyond the Pitch: Mastering Strategic Communic...,2025-03-06T11:47:28
...,...,...,...,...,...,...,...,...,...,...,...,...
174,ChiPy Algorithm SIG,https://www.meetup.com/_chipy_/events/vxxpjtyh...,2025-11-06 18:30:00,2025-11-06 20:15:00,TBD,Looking for a host,Chicago,il,ChiPy: Chicago's Official Python User Group,https://www.google.com/maps/search/?api=1&quer...,IMPORTANT NOTE: IN-PERSON event at To Be Deter...,2025-03-03 00:21:17.092864+00:00
175,Socratic Seminar,https://www.meetup.com/chibitdevs/events/jqtzl...,2025-11-13 18:00:00,2025-11-13 19:30:00,200 N La Salle St,200 N La Salle St,Chicago,IL,Chicago BITDEVS,https://www.google.com/maps/search/?api=1&quer...,Please note that Chicago BitDevs aims at a tec...,2025-03-03 00:21:30.924738+00:00
176,"Tech Pulse 2030: Emerging Technologies, Respon...",https://www.meetup.com/ai-2030-responsible-art...,2025-12-02 17:30:00,2025-12-02 19:30:00,1871,222 W Merchandise Mart Plaza #1212,Chicago,IL,AI 2030: Responsible Artificial Intelligence f...,https://www.google.com/maps/search/?api=1&quer...,Tech Pulse 2030 is more than an event series—i...,2025-03-03 00:17:12.403765+00:00
177,ChiPy Algorithm SIG,https://www.meetup.com/_chipy_/events/vxxpjtyh...,2025-12-04 18:30:00,2025-12-04 20:15:00,TBD,Looking for a host,Chicago,il,ChiPy: Chicago's Official Python User Group,https://www.google.com/maps/search/?api=1&quer...,IMPORTANT NOTE: IN-PERSON event at To Be Deter...,2025-03-06T11:47:21


In [7]:
upload_to_gsheets(combined_df, "Events",['eventURL'], verbose=True)

Existing data: (703, 12)
New data: (22, 12)


In [9]:
events_df = get_gsheet_df("Events")
create_event_markdown(events_df, "Chicago Events")

events_df = get_gsheet_df("Events")
create_discord_list(events_df, "Chicago Events")

ParserError: String does not contain a date: 